In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

from doctr.io import DocumentFile
from doctr.models import ocr_predictor
import easyocr
import json
import pandas as pd
from PIL import Image

import dvcx
from dvcx.query import C, DatasetQuery, udf
from dvcx.lib.dataset import Dataset

os.environ['AWS_PROFILE']='iterative-sandbox'
print(os.environ['AWS_PROFILE'])

# 1 - Create/Add a Dataset

In [ ]:
WILDRECEIPT_S3 = "s3://cse-dvcx-public/example-dvcx-ocr/"

In [ ]:
# List files 
ds = (DatasetQuery(path=WILDRECEIPT_S3)
      .filter(C.name.endswith(".jpeg"))
     )

ds.to_pandas()

In [ ]:
ds

# Date Pre-Processing

## Get Dataset Sample (filtering)

In [ ]:
ds_ocr = (ds
 # .filter(C.name.glob("*.jpeg"))
)
ds_ocr.to_pandas()

## Add metadata (from .txt)

In [ ]:
# from dvcx.query import udf, C, DatasetQuery
# from sqlalchemy import Integer


# @udf(
#     output={"path_len": Integer},
#     params=(C.parent, C.name),
#     batch=10,
# )
# def name_len(names):
#     print(type(names))
#     return [(len(parent + name),) for (parent, name) in names]


In [ ]:
# # Add a signal with funciton-based UDF
# ds_with_name_len = ds_ocr.add_signals(name_len)
# ds_with_name_len.to_pandas()

In [ ]:
ds = (
    Dataset(
        WILDRECEIPT_S3,
        # client_config={"aws_anon": True},
    )
    # .filter(C.name.glob("*.pdf"))
    .limit(10)
    # .map(PartitionObject(), parallel=False)
)
ds.to_pandas()

In [ ]:
from sqlalchemy import Integer

def count(parent):
    return (len(parent),)

ds.map(count, output={"length": Integer}).to_pandas()

In [ ]:
from sqlalchemy import Text

# Use `.map` 

def add_file_name(parent, name):
    path_in_medatata = []
    path = parent.strip('/') + '/' + name
    path = path.replace('example-dvcx-ocr/wildreceipt/', '')

    return (path,)

ds.map(add_file_name, output={"file_name": Text}).to_pandas()

In [ ]:
# # Use UDF


# from sqlalchemy import Text

# @udf(
#     output={"file_name": Text},
#     params=(C.parent, C.name),
#     batch=10,
# )
# def add_file_name(params):
#     path_in_medatata = []
    
    
#     for parent, name in params: 
#         # print(parent, name)
#         path = parent.strip('/') + '/' + name
#         path = path.replace('example-dvcx-ocr/wildreceipt/', '')
#         # print(path)
#         path_in_medatata.append( (path, ) )  # Why it should be a tuple here? 
#     return path_in_medatata 

# ds_with_name_len = ds_ocr.add_signals(add_file_name)
# ds_with_name_len.to_pandas()

In [ ]:
import json 

# File path
file_path = '../data/wildreceipt/train.txt'

# Read JSON data from file
meta = []
with open(file_path, 'r') as file:
    for line in file:
        meta.append(json.loads(line.strip()))

len(meta)

In [ ]:
# meta[0]

In [ ]:
# for file in meta[:100]:
#     if file.get('file_name', '') == 'image_files/Image_16/11/d5de7f2a20751e50b84c747c17a24cd98bed3554.jpeg':
#         print(file)

In [ ]:
# file.keys()

In [ ]:
from tqdm import tqdm

def add_annotations(file_name):

    # Read JSON data from file
    file_path = '../data/wildreceipt/train.txt'
    
    with open(file_path, 'r') as file:
        for line in file:
            meta = json.loads(line.strip())
            if meta.get('file_name', '') == file_name:
                # print(file_name)
                # res = file
                break

    return (meta['height'], meta['width'], str(meta['annotations']),)


ds = (Dataset(WILDRECEIPT_S3)
    .filter(C.name.glob("*.jpeg"))
    .limit(100)
    # .map(PartitionObject(), parallel=False)
)

ds = ds.map(add_file_name, output={"file_name": Text})
ds = ds.map(add_annotations, output={"height": Integer, "width": Integer, "annotations":Text})
ds.to_pandas()

In [ ]:
ds.save('wildreceipt-train')

In [ ]:
from dvcx.cli import ls_datasets, get_catalog

ls_datasets()

In [ ]:
cat = get_catalog()
cat.get_dataset('wildreceipt-train')

In [ ]:
ds_train = cat.get_dataset('wildreceipt-train')

In [ ]:
from dvcx.dataset import create_dataset_uri

create_dataset_uri('wildreceipt-train')

In [ ]:
ds = Dataset('ds://wildreceipt-train')
ds.to_pandas()

In [ ]:
# from sqlalchemy import Text

# @udf(
#     output=(
#         ("height", Integer),
#         # "width": Integer,
#         # "annotations": Text,
#     ),
#     params=(C.file_name),
#     batch=10,
# )
# def add_annotations(params):

#     # File path
#     file_path = '../data/wildreceipt/train.txt'
    
#     # Read JSON data from file
#     meta = []
#     with open(file_path, 'r') as file:
#         for line in file:
#             meta.append(json.loads(line.strip()))
#     print(len(meta))
    
#     metadata = [] 
#     for path in params: 
#         print(path)
        
#         for file in meta:

#             print(file)
            
#             if file.get('file_name', '') == path:
#                 print(path)
#                 metadata.append(
#                     (
#                         file.get('height', ''), 
#                         # file.get('width', ''), 
#                         # str(file.get('annotations', '')),
#                     )
#                 )
    
#     return metadata

# ds_with_name_meta = ds_with_name_len.add_signals(add_annotations)
# ds_with_name_meta.to_pandas()

In [ ]:
ds.to_pandas()

# Detect text - OCR

In [ ]:
import base64
import io
import os

import requests

try:
    from PIL import Image, ImageOps, UnidentifiedImageError
except ImportError as exc:
    raise ImportError(
        "Missing dependency Pillow for computer vision:\n"
        "To install run:\n\n"
        "  pip install 'dvcx[cv]'\n"
    ) from exc

from dvcx.query import Object, udf
from dvcx.sql.types import String

def encode_image(raw):
    try:
        img = Image.open(raw)
    except UnidentifiedImageError:
        return None
    img.load()
    # img = img.convert("RGB")
    # img = ImageOps.fit(img, DEFAULT_FIT_BOX)
    return img


@udf(
    params=(parent: Text,),  # Columns consumed by the UDF.
    output={
        "ocr": String,
    },  # Signals being returned by the UDF.
    method="process",
)
class OpenImageDetect:
    def __init__(self):
        self.model = ocr_predictor(det_arch='db_resnet50', reco_arch='crnn_vgg16_bn', pretrained=True)

    def process(self, ):

        print(img)

        # with stream_jpg.open() as fd:
        #     img = Image.open(fd)

        print(self.model)
        doc = DocumentFile.from_images(fd)
        result = model(doc)

        return (result, )

In [ ]:
ds = (Dataset(WILDRECEIPT_S3)
    .filter(C.name.glob("*.jpeg"))
    .limit(100)
    # .map(PartitionObject(), parallel=False)
)

ds = ds.map(add_file_name, output={"file_name": Text})
ds.to_pandas()

In [ ]:
def process(img):

    print(img)

    # with stream_jpg.open() as fd:
    #     img = Image.open(fd)

    print(self.model)
    doc = DocumentFile.from_images(fd)
    result = model(doc)

    return (result, )

ds = ds.map
    .aggregate(
        OpenImageDetect(),
        # partition_by=path.file_stem(C.name),
    )
)

ds.to_pandas()

## IMAGE → OCR Engine -> Text Boxes

In [ ]:
from doctr.io import DocumentFile
from doctr.models import ocr_predictor
import json


In [ ]:
model = ocr_predictor(det_arch='db_resnet50', reco_arch='crnn_vgg16_bn', pretrained=True)
model

In [ ]:
def load_image(raw):
    # img = Image.open(raw)
    # img.load()
    return img



## Post-Process / Clean 

# Augment Extractions with LLM

## Text boxes → LLM → JSON object